# See how well it did on real data
The spectra are real. We have results from `ssv` files from SpectrumMill. We can load this and see if the sequences we identified matches the sequences SpectrumMill identified, plus the hybrid ones

## 1. Load the `ssv` file

In [1]:
import pandas as pd
import json

specmilresultsfile = '/Users/zacharymcgrath/Downloads/NOD2_E3_results.ssv'
specmilresults = pd.read_csv(specmilresultsfile, sep=';')


In [2]:
specmilresults.loc[specmilresults['filename'] == 'NOD2_E3.7065.7065.2']

,number,filename,parent_charge,score,deltaForwardReverseScore,deltaRank1Rank2Score,percent_scored_peak_intensity,totalIntensity,previous_aa,sequence,next_aa,retentionTimeMin,chromatographicPeakWidthSec,parent_m_over_z,species,entry_name
6,7,NOD2_E3.7065.7065.2,2,12.55,6.28,7.14,81.3,282000.0,(G),DLPVNSPMTKG,(D),26.08,15.0,579.7953,MOUSE,HYBRID: mouse ins2C EVEDPQVAQLELGGGPGAGD-LPVNS...


### 1.a Get the hybrids out of the dataframe

In [3]:
specmilhybs = specmilresults.loc[specmilresults['entry_name'].str.contains('HYBRID')]
filenameindexedhybs = {row['filename']: row['sequence'] for _, row in specmilhybs.iterrows()}


### 1.b Get the non hybrids out of the dataframe

In [4]:
specmilnonhybs = specmilresults.loc[~specmilresults['entry_name'].str.contains('HYBRID')]
filenameindexednonhybs = {row['filename']: row['sequence'] for _, row in specmilnonhybs.iterrows()}

## 2. Load the results from hypedsearch

In [5]:
nonhybridresultsfile = '/Users/zacharymcgrath/Desktop/Experiment output/filtered_NOD2_E3/summary.tsv'
hybridresultsfile = '/Users/zacharymcgrath/Desktop/Experiment output/filtered_NOD2_E3/hybrid_summary.tsv'
resultsjsonfile = '/Users/zacharymcgrath/Desktop/Experiment output/filtered_NOD2_E3/summary.json'

nonhybridresults = pd.read_csv(nonhybridresultsfile, sep='\t')
hybridresults = pd.read_csv(hybridresultsfile, sep='\t')


In [6]:
nonhybridresults.head()

,Unnamed: 0,proteins,sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,['INS1_MOUSE Insulin-1'],DPQVEQLEL,6,2,9,0.000673,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.13446.13477.2.pkl
1,1,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",DLQTLALEVA,7,2,11,0.002875,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.18005.18246.2.pkl
2,2,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",DLQTLALE,5,3,10,0.000878,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.13729.13828.2.pkl
3,3,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",DLQTLAL,4,2,10,0.000438,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.15226.15503.2.pkl
4,4,['CMGA_MOUSE Chromogranin-A'],LPVNSPMTKGD,0,0,0,0.000452,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.7065.7065.2.pkl


In [7]:
hybridresults.head()

,Unnamed: 0,left_proteins,right_proteins,sequence,hybrid_sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",['SSRD_MOUSE Translocon-associated protein sub...,DLQTLALLL,DLQT-LALLL,6,2,10,0.000194,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.21510.21510.2.pkl
1,1,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",['PCX4_MOUSE Pecanex-like protein 4'],DLQTLALEFGLGG,DLQTLA-LEFGLGG,4,7,7,0.003455,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.16373.16401.2.pkl
2,2,['CLCB_MOUSE Clathrin light chain B'],[],SVLMSLNARAE,SVLMS(L)NARAE,1,0,1,1.975836,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.10635.10674.3.pkl
3,3,"['INS1_MOUSE Insulin-1', 'INS2_MOUSE Insulin-2']",['FYCO1_MOUSE FYVE and coiled-coil domain-cont...,DLQTLALELN,DLQTL-ALELN,7,0,9,0.496485,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.16920.17393.2.pkl
4,4,['MOFA1_MOUSE MORF4 family-associated protein 1'],['TCTP_MOUSE Translationally-controlled tumor ...,SEESALNHLQVEGKMVSRTE,SEESALNHLQ-VEGKMVSRTE,4,1,3,2.988467,/Users/zacharymcgrath/Desktop/nod2 data/filter...,NOD2_E3.3808.3850.4.pkl


## 3. Check to see the overlap in results

In [8]:
# pairs are (hypedsearch, specmil, id)
nonhybpairs = []
for _, row in nonhybridresults.iterrows():
    fname = row['id'].replace('.pkl', '')
    if fname in filenameindexednonhybs:
        nonhybpairs.append((row['sequence'], filenameindexednonhybs[fname], fname, 'non-hybrid'))
    else:
        nonhybpairs.append((row['sequence'], filenameindexedhybs[fname], fname, 'hybrid'))
        
hybpairs = []
for _, row in hybridresults.iterrows():
    fname = row['id'].replace('.pkl', '')
    if fname in filenameindexednonhybs:
        hybpairs.append((row['sequence'], filenameindexednonhybs[fname], fname, 'non-hybrid'))
    else:
        hybpairs.append((row['sequence'], filenameindexedhybs[fname], fname, 'hybrid'))



In [9]:
gc = 0
missed_nhpairs = []
for pair in nonhybpairs:
    # replace LI with B 
    hyped_LI_less = pair[0].replace('L', 'B').replace('I', 'B')
    specmil_LI_less = pair[1].replace('L', 'B').replace('I', 'B')
    if hyped_LI_less != specmil_LI_less:
        missed_nhpairs.append(pair)
        print(pair)
    else:
        gc += 1
print(f'Successfully aligned {gc}/{len(filenameindexednonhybs)} ')

('LPVNSPMTKGD', 'DLPVNSPMTKG', 'NOD2_E3.7065.7065.2', 'hybrid')
('LDTQLKPRNT', 'DLQTLALNAAR', 'NOD2_E3.10614.10681.2', 'hybrid')
('ELTNIELL', 'DLQTLEVE', 'NOD2_E3.11427.11516.2', 'hybrid')
('SAAPGFHQQLRL', 'DLQTLALEVAQQ', 'NOD2_E3.18217.18415.3', 'non-hybrid')
('SSTDQKESQPPEKTTED', 'ERGFFYTPMSRREVE', 'NOD2_E3.6434.6502.4', 'non-hybrid')
('DAIAELDTLSEE', 'DFTLTISSVQAE', 'NOD2_E3.16326.16479.2', 'non-hybrid')
('NSVEFMQIWRK', 'DIAAKYKELGFQG', 'NOD2_E3.8307.8394.3', 'non-hybrid')
('AVGSFSPSVTNS', 'DFIETYYLS', 'NOD2_E3.16911.16911.2', 'non-hybrid')
('QGGDDLDPNY', 'DIDGDGQVNY', 'NOD2_E3.8265.8265.2', 'non-hybrid')
('SGEHSIIGRTMVVHEKQDDLGKGGNE', 'DGTITTKELGTVMRSLGQNPTEAELQ', 'NOD2_E3.14735.14774.4', 'non-hybrid')
('ESQFEMD', 'DVNWGYE', 'NOD2_E3.10602.10740.2', 'non-hybrid')
('EEYTSTMM', 'GNPDDSFLE', 'NOD2_E3.9224.9311.2', 'non-hybrid')
('YVVFFF', 'VNPTVFF', 'NOD2_E3.14326.14398.2', 'non-hybrid')
('VSGAASLSPL', 'ELARFHK', 'NOD2_E3.1309.1309.2', 'non-hybrid')
('FSSFGPISEVVVV', 'DFPEEVAIAEEL', '

In [10]:
gch = 0
missed_hpairs = []
for pair in hybpairs:
    if pair[0] != pair[1]:
        missed_hpairs.append(pair)
        print(pair)
    else:
        gch += 1
print(f'Successfully aligned {gch}/{len(filenameindexedhybs)}')

('DLQTLALEFGLGG', 'DLQTLALWSRM', 'NOD2_E3.16373.16401.2', 'hybrid')
('SVLMSLNARAE', 'DLQTLALNAAR', 'NOD2_E3.10635.10674.3', 'hybrid')
('DLQTLALELN', 'DLQTLALEVE', 'NOD2_E3.16920.17393.2', 'hybrid')
('SEESALNHLQVEGKMVSRTE', 'DTGAGSIREAGGAFGKREKAEE', 'NOD2_E3.3808.3850.4', 'non-hybrid')
('QRCLNTLGSYMDLIEQVEK', 'DTGAGSIREAGGAFGKREKAEE', 'NOD2_E3.3820.3820.5', 'non-hybrid')
('LDEVKDYVLIPNASQPESKVFYLKMKG', 'DIYNFFSPLNPVRVHIEIGPDGRVTGEA', 'NOD2_E3.17254.17254.4', 'non-hybrid')
('KKLEAAEERRKSHEGVFAGR', 'DVTKGRKFYGPEGPYGVFAGR', 'NOD2_E3.7225.7225.4', 'non-hybrid')
('SSSSYRREYGKIDT', 'DSRPGGYGYGYGRSR', 'NOD2_E3.3190.3222.4', 'non-hybrid')
('TGTSATNNVNILGGSPG', 'EDPQVEQLELGGSPG', 'NOD2_E3.11823.11823.2', 'non-hybrid')
('TTKTYTPKSGF', 'ERGFFYTPKS', 'NOD2_E3.5323.5391.3', 'non-hybrid')
('DPQVAQLELGGSPALPPAANSSP', 'DPQVAQLELGGGPGAGDLQTLAL', 'NOD2_E3.21249.21249.3', 'non-hybrid')
('TDMLESTTLCPE', 'DYSLTISSLESE', 'NOD2_E3.16050.16050.2', 'non-hybrid')
('SFSFLLSPY', 'DFTPAVHASL', 'NOD2_E3.9820.9820.2'

## 4. See what the other alignments were for the missed sets

In [11]:
# load the json
jsres = json.load(open(resultsjsonfile, 'r'))

# all the entries are <filename>_<scan_no> so go through and index them by their filename
filenameindexedhypedsearch = {}
for name, alignments in jsres.items():
    # get the id
    fname = alignments['spectrum']['id'].replace('.pkl', '')
    
    filenameindexedhypedsearch[fname] = alignments


In [12]:
# for each of the unsuccesfull non hybrids, print the others
for nh in missed_nhpairs:
    print()
    print(f'Acutal alignment: {nh[1]} \t Hybrid: {"non" not in nh[3]} \t ID: {nh[2]}')
    print('Attempted alignemnts:')
    for alignment in filenameindexedhypedsearch[nh[2]]['alignments']:
        hybrid = 'hybrid_sequence' in alignment
        foundstring = '*' if alignment['sequence'] == nh[1] else ''
        print(f'{foundstring}Sequence: {alignment["sequence"]} \t Hybrid: {hybrid} \t Total score: {alignment["total_score"]} \t \
              B score: {alignment["b_score"]} \t Y score: {alignment["y_score"]}')


Acutal alignment: DLPVNSPMTKG 	 Hybrid: True 	 ID: NOD2_E3.7065.7065.2
Attempted alignemnts:
Sequence: LPVNSPMTKGD 	 Hybrid: False 	 Total score: 0 	               B score: 0 	 Y score: 0

Acutal alignment: DLQTLALNAAR 	 Hybrid: True 	 ID: NOD2_E3.10614.10681.2
Attempted alignemnts:
Sequence: LDTQLKPRNT 	 Hybrid: False 	 Total score: 4 	               B score: 2 	 Y score: 3

Acutal alignment: DLQTLEVE 	 Hybrid: True 	 ID: NOD2_E3.11427.11516.2
Attempted alignemnts:
Sequence: ELTNIELL 	 Hybrid: False 	 Total score: 10 	               B score: 5 	 Y score: 1
Sequence: ELDVEELP 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 0
Sequence: ELDVEEPN 	 Hybrid: True 	 Total score: 8 	               B score: 5 	 Y score: 0

Acutal alignment: DLQTLALEVAQQ 	 Hybrid: False 	 ID: NOD2_E3.18217.18415.3
Attempted alignemnts:
Sequence: SAAPGFHQQLRL 	 Hybrid: False 	 Total score: 0 	               B score: 0 	 Y score: 0

Acutal alignment: ERGFFYTPMSRREVE 	 Hybrid: False 	 ID: NO

In [13]:
for h in missed_hpairs:
    print()
    print(f'Acutal alignment: {h[1]} \t Hybrid: {"non" not in h[3]} \t ID: {h[2]}')
    print('Attempted alignments:')
    for alignment in filenameindexedhypedsearch[h[2]]['alignments']:
        hybrid = 'hybrid_sequence' in alignment
        foundstring = '*' if alignment['sequence'] == h[1] else ''
        print(f'{foundstring}Sequence: {alignment["sequence"]} \t Hybrid: {hybrid} \t Total score: {alignment["total_score"]} \t \
              B score: {alignment["b_score"]} \t Y score: {alignment["y_score"]}')


Acutal alignment: DLQTLALWSRM 	 Hybrid: True 	 ID: NOD2_E3.16373.16401.2
Attempted alignments:
Sequence: DLQTLALEFGLGG 	 Hybrid: True 	 Total score: 7 	               B score: 4 	 Y score: 7
Sequence: DLQTLALGQIFD 	 Hybrid: True 	 Total score: 6 	               B score: 4 	 Y score: 6
Sequence: DLQTLADATLSGK 	 Hybrid: True 	 Total score: 4 	               B score: 4 	 Y score: 0

Acutal alignment: DLQTLALNAAR 	 Hybrid: True 	 ID: NOD2_E3.10635.10674.3
Attempted alignments:
Sequence: SVLMSLNARAE 	 Hybrid: True 	 Total score: 1 	               B score: 1 	 Y score: 0

Acutal alignment: DLQTLALEVE 	 Hybrid: True 	 ID: NOD2_E3.16920.17393.2
Attempted alignments:
Sequence: DLQTLALELN 	 Hybrid: True 	 Total score: 9 	               B score: 7 	 Y score: 0
Sequence: AGDLQTLALEV 	 Hybrid: False 	 Total score: 2 	               B score: 1 	 Y score: 1
Sequence: GDLQTLALEVA 	 Hybrid: False 	 Total score: 0 	               B score: 0 	 Y score: 0

Acutal alignment: DTGAGSIREAGGAFGKREKAEE 	 Hybri